<a href="https://colab.research.google.com/github/thekirti02/Plant-disease/blob/main/ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tpot

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 31.0 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=f017e01bd1e05773c25d6b5b7b4b188f81d62b62d4f57565c50d9583f766819a
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier

# Define the path to your dataset
folder_path = '/content/drive/MyDrive/Plant_dataset'

# Lists to store preprocessed images and labels
images = []
labels = []

# Define image size and preprocessing steps
image_size = (4, 2)  # Adjust size as needed
resize_method = Image.BICUBIC

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Counters for healthy and rust images
healthy_count = 0
rust_count = 0

# Preprocess images and labels
for filename in os.listdir(folder_path):
    if filename.startswith('Healthy'):
        label = 'Healthy'
        healthy_count += 1
    elif filename.startswith('Rust'):
        label = 'Rust'
        rust_count += 1
    else:
        continue

    img = Image.open(os.path.join(folder_path, filename))
    img = img.resize(image_size, resample=resize_method)
    img_vector = np.array(img).flatten()[:8]  # Convert to vector and limit to 8 features

    # Append the image vector and label to respective lists
    images.append(img_vector)
    labels.append(label)

# Convert to arrays
X = np.array(images)
y = label_encoder.fit_transform(labels)

# Normalize images
X = X / 255.0  # Normalize pixel values to range [0, 1]

# Split into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.3, random_state=42)

# Use TPOTClassifier for AutoML
tpot = TPOTClassifier(verbosity=2, generations=10, population_size=50, random_state=42)


# Train the model
tpot.fit(train_images, train_labels)

# Evaluate the model
score = tpot.score(test_images, test_labels)
print(f"Accuracy: {score}")

# Export the best pipeline
tpot.export('best_pipeline.py')


Optimization Progress:   0%|          | 0/550 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8274412698412699

Generation 2 - Current best internal CV score: 0.8274412698412699

Generation 3 - Current best internal CV score: 0.8274412698412699

Generation 4 - Current best internal CV score: 0.8274412698412699

Generation 5 - Current best internal CV score: 0.8274412698412699

Generation 6 - Current best internal CV score: 0.8338412698412698

Generation 7 - Current best internal CV score: 0.8338412698412698

Generation 8 - Current best internal CV score: 0.8338412698412698

Generation 9 - Current best internal CV score: 0.8338412698412698

Generation 10 - Current best internal CV score: 0.8338412698412698

Best pipeline: ExtraTreesClassifier(ZeroCount(input_matrix), bootstrap=True, criterion=entropy, max_features=0.55, min_samples_leaf=1, min_samples_split=2, n_estimators=100)
Accuracy: 0.8327137546468402
